<a href="https://colab.research.google.com/github/PaulaPineroCouceiro/deeplearning/blob/main/Ejercicio%20Regularizadores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio con regularizadores

In [10]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
print(tf.keras.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))
print(tf.__version__)
from keras.datasets import mnist
from keras import models
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
from keras import regularizers

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
2.9.0
GPU Available: []
2.9.2


In [11]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data() # Descarga los datos de MNIST

11490434/11490434 [==============================] - 0s 0us/step


In [13]:
# Prepara con reshape y normaliza las imágenes tanto de entrenamiento como de test
train_images = train_images.reshape(60000,28*28)
test_images = test_images.reshape((10000,28*28))
 
train_images = train_images.astype('float32')/ 255
test_images = test_images.astype('float32')/255


In [14]:
# Crear un subconjunto de imágenes de entrenamiento que llamaré
# validation_images, para hacer las validaciones, con las 10000 últimas imágenes 
# del set de datos de entrenamiento (train_images) (obviamente, lo mismo con las
# etiquetas correspondientes -> validation_labels). Deja preparadas las 50K 
# primeras imágenes y etiquetas para entrenamiento (train_images y train_labels)
partial_train_images=train_images[:50000]
validation_images =train_images[50000:]
partial_train_labels= train_labels[:50000]
validation_labels=train_labels[50000:]

In [33]:
def crea_modelo(regularizador):
  # Define una función que recibe un número del 0 al 4 que creará:
  # un modelo secuencial con una capa de entrada densa de 512 neuronas
  # con activación relu y la capa de salida que le corresponda según el problema
  # Ahora, en función del número del regularizador:
  # Si es 0, no mete ningún regularizador
  # Si es 1, mete un regularizador de tipo l1 y valor 0.001
  # Si es 2, mete un regularizador de tipo l2 y valor 0.001
  # Si es 3, mete un regularizador de tipo l1_l2 y valor 0.001 para ambos
  # Si es 4, mete un Dropout del 50% después de la primera capa densa
  # Si es distinto que estos valores, escribe el error y retorna
  # Al final, esta función pinta el summary del modelo y retorna el modelo creado
    modelo = models.Sequential()
    
    if regularizador==0:
       modelo.add(layers.Dense(512, activation='relu', input_shape=(28*28,)))
       modelo.add(layers.Dense(10, activation='softmax'))
       modelo.summary()
    elif regularizador==1:
       modelo.add(layers.Dense(512, kernel_regularizer=regularizers.l1(0.001), activation='relu',
                           input_shape=(28*28,)))
       modelo.add(layers.Dense(10, activation='softmax'))
       modelo.summary()
    elif regularizador==2:
      modelo.add(layers.Dense(512, kernel_regularizer=regularizers.l2(0.001), activation='relu',
                           input_shape=(28*28,)))
      modelo.add(layers.Dense(10, activation='softmax'))
      modelo.summary()
    elif regularizador==3:
      modelo.add(layers.Dense(512, kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.002), activation='relu',
                           input_shape=(28*28,)))
      modelo.add(layers.Dense(10, activation='softmax'))
      modelo.summary()
    elif regularizador==4:
      modelo.add(layers.Dense(512, activation='relu', input_shape=(28*28,)))
      modelo.add(layers.Dropout(0.5))
      modelo.add(layers.Dense(10, activation='softmax'))
      modelo.summary()
    else:
      print('Error no es ninguno de estos valores \n ')
     
    return modelo

   


In [38]:
def compila_modelo(modelo): 
  # Define una función que recibe un modelo, lo compila con un optimizador
  # 'rmsprop', con una función de pérdida 'sparse_categorical_crossentropy' y
  # una métrica de precisión. La función retorna el modelo
    modelo.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    return modelo   


In [39]:
def entrena_modelo(modelo):
  # Define una función que entrena el modelo que se le pasa por parámetro,
  # con las imágenes y etiquetas de entrenamiento, con 30 iteraciones y 128
  # de tamaño de minilote y con el validation_data creado más arriba.
  # Esta función devuelve un history
    historia0=modelo.fit(train_images, train_labels, epochs=30, batch_size=128,validation_data=(validation_images,validation_labels))
    return historia0

In [40]:
def pinta_perdida_entrenamiento_validacion(historia):
  history_dict = historia.history
  acc = historia.history['accuracy']
  val_acc = historia.history['val_accuracy']
  loss = historia.history['loss']
  val_loss = historia.history['val_loss']
  epochs = range(1, len(acc) + 1)
  # "bo" es para "punto azun"
  plt.plot(epochs, loss, 'bo', label='Pérdida de Entrenamiento')
  # "b" es para "linea continua azul"
  plt.plot(epochs, val_loss, 'b', label='Pérdida de Validación')
  plt.title('Pérdida de Entrenamiento y Validación')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()
  plt.show()

In [41]:
def pinta_precision_entrenamiento_validacion(historia):
  history_dict = historia.history
  acc = historia.history['accuracy']
  val_acc = historia.history['val_accuracy']
  loss = historia.history['loss']
  val_loss = historia.history['val_loss']
  plt.clf()   # Limpia la figura
  acc_values = history_dict['accuracy']
  val_acc_values = history_dict['val_accuracy']
  epochs = range(1, len(acc) + 1)
  plt.plot(epochs, acc, 'bo', label='Precisión de Entrenamiento')
  plt.plot(epochs, val_acc, 'b', label='Precisión de Validación')
  plt.title('Precisión de Entrenamiento y Valización')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()
  plt.show()

## Una vez definidas todas las funciones anteriores, ejecuta las siguientes llamadas a dichas funciones y deduce que está pasando y, para este contexto, que es lo que mejor está funcionando

### Caso 0, sin regularizadores

In [42]:
network0 = crea_modelo(0)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [43]:
network0 = compila_modelo(network0)

In [44]:
historia0 = entrena_modelo(network0)

Epoch 1/30
469/469 [==============================] - 6s 12ms/step - loss: 0.2551 - accuracy: 0.9260 - val_loss: 0.1139 - val_accuracy: 0.9692
Epoch 2/30
469/469 [==============================] - 5s 11ms/step - loss: 0.1031 - accuracy: 0.9690 - val_loss: 0.0755 - val_accuracy: 0.9787
Epoch 3/30
469/469 [==============================] - 5s 11ms/step - loss: 0.0689 - accuracy: 0.9792 - val_loss: 0.0479 - val_accuracy: 0.9846
Epoch 4/30
469/469 [==============================] - 6s 12ms/step - loss: 0.0495 - accuracy: 0.9851 - val_loss: 0.0393 - val_accuracy: 0.9890
Epoch 5/30
469/469 [==============================] - 6s 12ms/step - loss: 0.0379 - accuracy: 0.9883 - val_loss: 0.0257 - val_accuracy: 0.9931
Epoch 6/30
469/469 [==============================] - 5s 12ms/step - loss: 0.0288 - accuracy: 0.9916 - val_loss: 0.0173 - val_accuracy: 0.9959
Epoch 7/30
469/469 [==============================] - 5s 12ms/step - loss: 0.0212 - accuracy: 0.9937 - val_loss: 0.0151 - val_accuracy: 0.9961

In [ ]:
pinta_perdida_entrenamiento_validacion(historia0)

In [ ]:
pinta_precision_entrenamiento_validacion(historia0)

### Caso 1: Con regularizador l1

In [ ]:
network1 = crea_modelo(1)

In [ ]:
network1 = compila_modelo(network1)

In [ ]:
historia1 = entrena_modelo(network1)

In [ ]:
pinta_perdida_entrenamiento_validacion(historia1)

In [ ]:
pinta_precision_entrenamiento_validacion(historia1)

### Caso 2: Con regularizador l2

In [ ]:
network2 = crea_modelo(2)

In [ ]:
network2 = compila_modelo(network2)

In [ ]:
historia2 = entrena_modelo(network2)

In [ ]:
pinta_perdida_entrenamiento_validacion(historia2)

In [ ]:
pinta_precision_entrenamiento_validacion(historia2)

### Caso 3: Con regularizador l1_l2

In [ ]:
network3 = crea_modelo(3)

In [ ]:
network3 = compila_modelo(network3)

In [ ]:
historia3 = entrena_modelo(network3)

In [ ]:
pinta_perdida_entrenamiento_validacion(historia3)

In [ ]:
pinta_precision_entrenamiento_validacion(historia3)

### Caso 4: Con Droput

In [ ]:
network4 = crea_modelo(4)

In [ ]:
network4 = compila_modelo(network4)

In [ ]:
historia4 = entrena_modelo(network4)

In [ ]:
pinta_perdida_entrenamiento_validacion(historia4)

In [ ]:
pinta_precision_entrenamiento_validacion(historia4)

### Caso 5: Error de código de regularizador

In [ ]:
network5 = crea_modelo(5)

Error: Código de regularizador debe estar entre 0 y 4


### CONCLUSIONES